# Möbius‑Digit‑Thread Scanner (v9b) — **gap_scale 20, spacing 5**

Rules carried over from earlier best‑run (v6):

| knob | value |
|------|-------|
| gap_scale | **20** |
| spacing   | **5** bright‑1’s between gaps |
| bright tail | 400 ones, ~200 gaps |
| tiny exponent *k* | **4** (subtract α⁻¹ / 10⁴) |
| φ‑scale grid | 0.003 944 0 ± 3 × 10⁻⁷ (step 1 × 10⁻⁷, 7 pts) |
| seed bits | 64 and 128 |

Diagnostics implemented exactly as before  

* digit‑thread detector (π, φ, e, γ, δ, √2)  
* simple 3‑power proximity (9, 27, 81, 127, 729)  
* raw Fibonacci closeness check

The table prints the 40 best rows sorted by *rel_err*.  
Aiming for **< 1 × 10⁻⁴** to match or beat the v6 record.

In [1]:
import mpmath as mp, pandas as pd, itertools, re
mp.mp.dps=260
ALPHA_INV=mp.mpf('137.035999084'); ALPHA=1/ALPHA_INV
PHI=(1+mp.sqrt(5))/2

In [2]:
def digits_of(num,n=700):
    s=str(num).replace('.','')[1:]
    return (s+'0'*n)[:n]
CONST_BANK={n:digits_of(v) for n,v in {
    'pi':mp.pi,'phi':PHI,'e':mp.e,'gamma':mp.euler,
    'delta':mp.mpf('4.669201609'),'sqrt2':mp.sqrt(2)}.items()}

In [3]:
def primes_upto(n):
    sieve=[True]*(n+1); sieve[0:2]=[False,False]
    for i in range(2,int(n**0.5)+1):
        if sieve[i]: sieve[i*i:n+1:i]=[False]*len(range(i*i,n+1,i))
    return [i for i,v in enumerate(sieve) if v]

def prime_gaps(count):
    ps=primes_upto(3000)
    return [ps[i+1]-ps[i] for i in range(count)]

In [4]:
def build_tail(bright_len=400,spacing=5,gap_scale=20):
    gaps=[round(g/gap_scale,4) for g in prime_gaps(bright_len//spacing)]
    tail=[]; gi=iter(gaps)
    for i in range(bright_len):
        tail.append(1)
        if (i+1)%spacing==0:
            tail.append(next(gi,1))
    return tail

def cf_to_real(coeffs):
    v=mp.mpf(0)
    for a in reversed(coeffs):
        v=1/(a+v)
    return v

In [5]:
def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n: a,b=b,b+a
    return b[:n]

def fib_fraction(bits):
    return mp.fsum(int(b)*mp.mpf(2)**(-(k+1)) for k,b in enumerate(fib_word_bits(bits)))

In [6]:
def breath(x,tiny,phi_scale):
    x=phi_scale*PHI/x
    x-=tiny
    x=1/x/9
    x=1/x
    return x/9

In [7]:
powers=[9,27,81,127,729]

def const_thread(xdig):
    for name,bank in CONST_BANK.items():
        for L in range(8,3,-1):
            for i in range(len(xdig)-L+1):
                sub=xdig[i:i+L]
                if sub in bank: return f'{name}:{sub}'
    return '-'

def near_fib(x,bits=(64,128)):
    for b in bits:
        if abs(x-fib_fraction(b))<1e-5:
            return True
    return False

In [8]:
tail=build_tail()
seed=cf_to_real(tail)
tiny=ALPHA_INV/1e4
phi_center=0.003944
phi_vals=[phi_center+i*1e-7 for i in range(-3,4)]      # ±3e-7

rows=[]
for bits in (64,128):
    for phi in phi_vals:
        x=breath(seed+tiny,tiny,phi)
        rel=float(abs((x-ALPHA)/ALPHA))
        xdig=str(x)[2:402]
        rows.append(dict(bits=bits,phi=phi,rel_err=rel,
                         thread=const_thread(xdig),
                         pow3=any(abs(float(x)-p)/p<0.005 or abs(float(1/x)-p)/p<0.005 for p in powers),
                         fib_near=near_fib(float(x))))
df=pd.DataFrame(rows).sort_values('rel_err').head(40)
df

,bits,phi,rel_err,thread,pow3,fib_near
6,64,0.003944,1.482100,pi:2097,False,False
13,128,0.003944,1.482100,pi:2097,False,False
12,128,0.003944,1.482136,pi:02841,False,False
5,64,0.003944,1.482136,pi:02841,False,False
4,64,0.003944,1.482171,pi:7093,False,False
11,128,0.003944,1.482171,pi:7093,False,False
10,128,0.003944,1.482207,pi:9334,False,False
3,64,0.003944,1.482207,pi:9334,False,False
9,128,0.003944,1.482242,pi:02701,False,False
2,64,0.003944,1.482242,pi:02701,False,False
